In [49]:
from pathlib import Path
import os
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()


@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic("bash", "", cell.format(**globals()))

In [50]:
tmp_dir = Path("../tmp/cobra_annotation")
data_dir = Path("../data/3Ftx")

cobra_file = data_dir / "cobra.cif"
cobra_annotation = tmp_dir / "cobra_annotation.toml"
cobra_svg = tmp_dir / "cobra.svg"

os.makedirs(tmp_dir, exist_ok=True)

In [51]:
import gemmi
from typing import List, Tuple
import numpy as np


def compute_cystine_bridges(structure_path: Path) -> List[Tuple[int, int]]:
    """
    Compute cystine bridges from a protein structure file.

    This function identifies disulfide bonds between cysteine residues
    by analyzing the distance between sulfur atoms.

    Args:
        structure_path: Path to the structure file (CIF format)

    Returns:
        List of tuples containing residue indices of cysteine pairs forming disulfide bonds
    """
    # Load the structure using gemi
    structure = gemmi.read_structure(
        str(structure_path), merge_chain_parts=True, format=gemmi.CoorFormat.Mmcif
    )

    # Extract all cysteine residues
    cysteines = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.name == "CYS":
                    # Find the sulfur atom (SG) in each cysteine
                    for atom in residue:
                        if atom.name == "SG":
                            cysteines.append(
                                (
                                    int(residue.seqid.num),
                                    str(chain.name),
                                    np.array(atom.pos.tolist()),
                                )
                            )
                            break

    # Identify disulfide bonds based on distance between sulfur atoms
    # Typical S-S bond distance is around 2.05 Å
    disulfide_threshold = 2.3  # Angstroms
    bridges = []

    for i in range(len(cysteines)):
        for j in range(i + 1, len(cysteines)):
            res_i, chain_i, pos_i = cysteines[i]
            res_j, chain_j, pos_j = cysteines[j]

            # Calculate distance between sulfur atoms
            distance = np.linalg.norm(pos_i - pos_j)

            if distance <= disulfide_threshold:
                bridges.append((res_i, chain_i, res_j, chain_j))

    return bridges


In [52]:
from pathlib import Path
from typing import List, Tuple
import toml


def create_cystine_bridge_annotations(
    bridges: List[Tuple[int, str, int, str]], output_path: Path
) -> None:
    """Create a TOML annotation file for cystine bridges.

    This function takes a list of cystine bridges and creates a TOML annotation file
    that can be used with flatprot's annotation system to visualize the disulfide bonds.

    Args:
        bridges: List of tuples containing residue pairs forming disulfide bonds
        output_path: Path where the TOML file should be saved

    Example:
        >>> bridges = [(3, 25), (6, 13), (18, 43)]
        >>> create_cystine_bridge_annotations(bridges, Path("cystine_bridges.toml"))
    """
    annotations = []

    for i, (res1, chain1, res2, chain2) in enumerate(bridges, 1):
        annotation = {
            "label": f"Cys Bridge {i}",
            "type": "line",
            "indices": [res1, res2],
            "chain": chain1,
        }
        annotations.append(annotation)

    toml_content = {"annotations": annotations}

    # Write the TOML file
    with open(output_path, "w") as f:
        toml.dump(toml_content, f)


In [53]:
# Compute cystine bridges in the cobra structure
cystine_bridges = compute_cystine_bridges(cobra_file)
print(f"Found {len(cystine_bridges)} cystine bridges:")
for bridge in cystine_bridges:
    print(f"  Cys{bridge[0]} - Cys{bridge[1]}")

create_cystine_bridge_annotations(cystine_bridges, cobra_annotation)
print(f"Cystine bridge annotations saved to {cobra_annotation}")

Found 5 cystine bridges:
  Cys3 - CysA
  Cys6 - CysA
  Cys18 - CysA
  Cys49 - CysA
  Cys65 - CysA
Cystine bridge annotations saved to ../tmp/cobra_annotation/cobra_annotation.toml


In [54]:
%%pybash

uv run flatprot project {cobra_file} -o {cobra_svg} --annotations {cobra_annotation}

2025-03-27 16:09:06 INFO     Using default styles                               
2025-03-27 16:09:06 INFO     Loaded 5 annotations from                          
                             ../tmp/cobra_annotation/cobra_annotation.toml      
2025-03-27 16:09:06 INFO     SVG saved to ../tmp/cobra_annotation/cobra.svg     
2025-03-27 16:09:06 INFO     Successfully processed structure:                  
2025-03-27 16:09:06 INFO       Structure file: ../data/3Ftx/cobra.cif           
2025-03-27 16:09:06 INFO       Output file: ../tmp/cobra_annotation/cobra.svg   
2025-03-27 16:09:06 INFO       Transformation: Inertia-based                    
2025-03-27 16:09:06 INFO       Annotations file:                                
                             ../tmp/cobra_annotation/cobra_annotation.toml      


In [55]:
from IPython.display import display, HTML

with open(cobra_svg, "r") as f:
    svg_content = f.read()

# Modify SVG to constrain its width
svg_content = svg_content.replace("<svg ", f'<svg style="width:100%; height:auto;" ')

html = f"""
<div style=" border: 1px solid black; text-align: center; border-radius: 20% 0 20% 0">
    <h3>Cobra</h3>
    {svg_content}
</div>
"""

html += "</div>"
display(HTML(html))
